In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import math
import json
import tensorflow.python.keras as keras
from keras.layers import Input, Layer, LeakyReLU, BatchNormalization, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Add, Lambda
from keras.models import Model, load_model
from keras.optimizers import SGD, Adam
import keras.backend as K
from PIL import Image
import os
'''
# file h5 dibuat dengan convert.py dari https://github.com/awe777/keras-yolo3
model0 = load_model("yolov3_tiny.h5")   # dibuat dari kode sebelum fork
model1 = load_model("yolov3_tiny_.h5")  # dibuat dari kode setelah diubah di fork pribadi
print(model0.to_json() == model1.to_json())
print(model.summary())
# '''
print()

Using TensorFlow backend.


In [2]:
def roundingAlgo(x): 
    # simplification of the first algo loop by simplifying the expression for range(1,7)
    # problem - rounding function is still slow = 2,5 hours / epoch
    # all non-speed problem of first algo still applies
    result = x - 0.5
    resultCos = K.cos(2 * math.pi * x)
    return result + K.sin(2 * math.pi * x) * (1 + resultCos) * (13 + 2 * resultCos - 18 * K.pow(resultCos, 2) - 32 * K.pow(resultCos, 3) + 80 * K.pow(resultCos, 4)) / 15

class RoundClampQ7_12(Layer):
    def __init__(self, **kwargs):
        super(RoundClampQ7_12, self).__init__(**kwargs)
        self.trainable = False
    def build(self, input_shape):
        super(RoundClampQ7_12, self).build(input_shape)
    def call(self, X):
        return K.clip(roundingAlgo(X * 4096), -524288, 524287) / 4096.0
    def get_config(self):
        # config = {"name": self.__class__.__name__}
        base_config = super(RoundClampQ7_12, self).get_config()
        return dict(list(base_config.items()))
class RoundOverflowQ7_12(Layer):
    def __init__(self, **kwargs):
        super(RoundOverflowQ7_12, self).__init__(**kwargs)
        self.trainable = False
    def build(self, input_shape):
        super(RoundOverflowQ7_12, self).build(input_shape)
    def call(self, X):
        return (((roundingAlgo(X * 4096) + 524288) % 1048576) - 524288) / 4096.0
    def get_config(self):
        # config = {"name": self.__class__.__name__}
        base_config = super(RoundOverflowQ7_12, self).get_config()
        return dict(list(base_config.items()))
class RoundClampQ3_4(Layer):
    def __init__(self, **kwargs):
        super(RoundClampQ3_4, self).__init__(**kwargs)
        self.trainable = False
    def build(self, input_shape):
        super(RoundClampQ3_4, self).build(input_shape)
    def call(self, X):
        return K.clip(roundingAlgo(X * 16), -128, 127) / 16.0
    def get_config(self):
        # config = {"name": self.__class__.__name__}
        base_config = super(RoundClampQ3_4, self).get_config()
        return dict(list(base_config.items()))
class RoundOverflowQ3_4(Layer):
    def __init__(self, **kwargs):
        super(RoundOverflowQ3_4, self).__init__(**kwargs)
        self.trainable = False
    def build(self, input_shape):
        super(RoundOverflowQ3_4, self).build(input_shape)
    def call(self, X):
        return (((roundingAlgo(X * 16) + 128) % 256) - 128) / 16.0
    def get_config(self):
        # config = {"name": self.__class__.__name__}
        base_config = super(RoundOverflowQ3_4, self).get_config()
        return dict(list(base_config.items()))
class Identity(Layer):
    def __init__(self, **kwargs):
        super(Identity, self).__init__(**kwargs)
        self.trainable = False
    def call(self, X):
        return X
    def get_config(self):
        # config = {"name": self.__class__.__name__}
        base_config = super(Identity, self).get_config()
        return dict(list(base_config.items()))
class IdentityFinalLayer(Layer):
    def __init__(self, **kwargs):
        super(IdentityFinalLayer, self).__init__(**kwargs)
        self.trainable = False
    def call(self, X):
        return X
    def get_config(self):
        # config = {"name": self.__class__.__name__}
        base_config = super(IdentityFinalLayer, self).get_config()
        return dict(list(base_config.items()))
    
def DBL(previousLayer, layerFilter, kernelSize=(3, 3), roundingFunction=Identity, name=None):
    placeholder = ""
    if name is not None:
        placeholder = str(name)
    else:
        placeholder = str(time.time_ns())
    return roundingFunction(name="SecondRound_"+placeholder)(LeakyReLU(alpha=0.1)(roundingFunction(name="FirstRound_"+placeholder)(BatchNormalization(name="BatchNorm_"+placeholder)(Conv2D(filters=layerFilter, kernel_size=kernelSize, padding='same', name="Conv2D_"+placeholder)(previousLayer)))))

customObjects = {
    'Identity': Identity,
    'RoundClampQ3_4': RoundClampQ3_4,
    'RoundOverflowQ3_4': RoundOverflowQ3_4,
    'RoundClampQ7_12': RoundClampQ7_12,
    'RoundOverflowQ7_12': RoundOverflowQ7_12,
    'IdentityFinalLayer': IdentityFinalLayer
}

In [3]:
def prepare_image(image_dir, input_shape):
    # from PIL import Image
    # import numpy as np
    image_data = 0
    h, w = input_shape
    try:
        image = Image.open(image_dir)
        iw, ih = image.size
        scale = min(w/iw, h/ih)
        nw = int(iw*scale)
        nh = int(ih*scale)
        dx = (w-nw)//2
        dy = (h-nh)//2
        image = image.resize((nw,nh), Image.BICUBIC)
        new_image = Image.new('RGB', (w,h), (128,128,128))
        new_image.paste(image, (dx, dy))
        image_data = np.array(new_image)/255.
        return image_data
    except:
        return None
# TO DO: implement yolo_head in order to get human-understandable result from model output

In [4]:
try:
    print("Model loading successful")
    loadedModel.load_weights("./saved_models/model_4_trainModel.h5", by_name=True, skip_mismatch=True)
    print("Model weight loading attempt successful")
except Exception as e:
    print("Failed to load model/weight data:", e)

KeyboardInterrupt: 